In [1]:
conf.set("spark.cores.max", 16)
conf.set("spark.cassandra.input.split.size_in_mb", 32)
conf.set("spark.cassandra.splitCount", 16)
conf.getAll()

[('spark.dirver.maxResultSize', '4g'),
 ('spark.cassandra.input.split.size_in_mb', '32'),
 ('spark.cores.max', '16'),
 ('spark.driver.memory', '16g'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.cassandra.connection.host', '10.224.12.32'),
 ('spark.master', 'spark://tpedevapp0265.d010.digitalriverws.net:7077'),
 ('spark.jars',
  'file:///drhome/ssetiadi/.ivy2/jars/databricks_spark-sklearn-0.2.3.jar,file:///drhome/ssetiadi/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///drhome/ssetiadi/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///drhome/ssetiadi/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///drhome/ssetiadi/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///drhome/ssetiadi/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///drhome/ssetiadi/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///drhome/ssetiadi/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///drhome/ssetiadi/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///drhome/ss

In [17]:
date_list

['20180101',
 '20180102',
 '20180103',
 '20180104',
 '20180105',
 '20180106',
 '20180107',
 '20180108',
 '20180109',
 '20180110',
 '20180111',
 '20180112',
 '20180113',
 '20180114',
 '20180115',
 '20180116',
 '20180117',
 '20180118',
 '20180119',
 '20180120',
 '20180121',
 '20180122',
 '20180123',
 '20180124',
 '20180125',
 '20180126',
 '20180127',
 '20180128',
 '20180129',
 '20180130']

In [2]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/drhdev/workspace/ml/'
start_date = '2018-01-01'
end_date = '2018-06-30'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        yield date_formatted

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   

def merge_by_sub(group):

    first = group[ group['renew_att_num'] == '1' ]
    
    if first.empty:
        group['first_attempt_date'] = ''
        group['first_day_of_month'] = 0
        group['first_transaction_hour'] = ''
        group['first_response_code'] = ''
        group['first_response_message'] = ''
        group['first_response_group'] = ''
        group['first_decline_type'] = ''
        group['days_between'] = 0
        return group
    
    last = group[ group['renew_att_num'] == group['renew_att_num'].max() ]
        
    first_attempt_date = first['transaction_date_in_string'].iloc[0]
    last_attempt_date = last['transaction_date_in_string'].iloc[0]
     
    
    last['first_attempt_date'] = first_attempt_date
    last['first_day_of_month'] = first['day_of_month'].iloc[0].astype(int)
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['first_transaction_hour'] = first['transaction_hour'].iloc[0]
    last['first_response_code'] = first['response_code'].iloc[0]
    
    first_response_msg = first['response_message'].iloc[0]
    last['first_response_message'] = first_response_msg  
    last['first_response_group'] =   first['response_message'].apply(group_response_msg).iloc[0]
    
    last['first_decline_type'] = decline_type(first_response_msg)
    last['days_between'] =  days_between(first_attempt_date, last_attempt_date)
   

    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)


# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)

        

date_list = list(date_range_condition_values(start_date, end_date))

rdd = sc \
    .cassandraTable("subs", "cpg_transaction", split_count=64, split_size=100000)
    
#     .where("received_date in ?", date_list)
    
#     .spanBy('received_date')

sdf = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="cpg_transaction", keyspace="subs")\
    .load()    
    
udfValueToStatus = udf(convert_status, IntegerType())  
df = sqlContext.createDataFrame(rdd, sdf.schema)

   
sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed'))")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

grouped_schema = sdf.schema.add("first_attempt_date", StringType())\
                            .add("first_day_of_month", IntegerType())\
                            .add("first_transaction_hour", StringType())\
                            .add("first_response_code", StringType())\
                            .add("first_response_message", StringType())\
                            .add("first_response_group", StringType())\
                            .add("first_decline_type", StringType())\
                            .add("days_between", IntegerType()) 

@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

pdf_all = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id")\
    .apply(merge_by_sub_spark)\
    .filter('days_between > 0')\
    .toPandas()
    
print("# total time:", time.time() - time_start)    
    
pdf_all

# total time: 10013.123660564423


,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,...,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,...,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[0...,do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,...,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,...,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,...,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,...,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,...,Authorize,0,2018-04-19 01:07:15,19,01,27054,Attempt lower amount.[51] [] [111800] [111800],attempt_lower_amount,Insufficient Funds,30
6,20180519,adbehru,10002261401,49863a4a-d046-4df8-93f4-dbe2612211df,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,...,Authorize,0,2018-04-19 01:07:15,19,01,27054,Attempt lower amount.[51] [] [111800] [111800],attempt_lower_amount,Insufficient Funds,30
7,20180519,adbehru,10002261401,4b0ebdb1-dbad-462a-bff7-e656c4c6f0c1,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,...,Authorize,0,2018-04-19 01:07:15,19,01,27054,Attempt lower amount.[51] [] [111800] [111800],attempt_lower_amount,Insufficient Funds,30
8,20180519,adbehru,10002261401,61b2f414-1027-40d8-9cb9-4a2e0fee8d12,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,...,Authorize,0,2018-04-19 01:07:15,19,01,27054,Attempt lower amount.[51] [] [111800] [111800],attempt_lower_amount,Insufficient Funds,30
9,20180419,avast,10002779101,1528c8cc-89ae-405a-a657-dece1eeaf8c9,None,NON3DS,"Citibank, National Association",AE,492082,Visa,...,Authorize,0,2018-04-04 01:09:51,4,01,27001,Transaction refused[05] [000000] [] [],txn_refused,txn_refused,15


In [9]:
# pdf_all['day_of_week'].value_counts(dropna=False)
# pdf_all.to_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv')
from IPython.display import display
pd.options.display.max_columns = None
display(pdf_all)

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,18,wednesday,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[0...,None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[0...,do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,None,None,18,wednesday,None,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.018051,BRL,MasterCard,netgiro-br,3,99,29.018051,BRL,46969296500,27001,Transaction refused[62] [null] [] [],None,New,13259159000,,2018-04-18 06:02:11,2018-04-18 01:02:11,01,6681601083,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,18,wednesday,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0620,None,None,25,wednesday,None,Credit,BR,DRI*BitDefender,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,229.99,66.742732,BRL,MasterCard,pagador,3,229.99,66.742732,BRL,47202743400,530,7: Do Not Honor,None,New,13259627500,,2018-04-25 06:01:35,2018-04-25 01:01:35,01,6706709733,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,18,wednesday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,None,19,saturday,None,Debit,RU,DRI*Adobe Systems,1411163460-R

In [3]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/drhdev/workspace/ml/'
start_date = '2018-01-01'
end_date = '2018-06-30'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        yield date_formatted

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   

def merge_by_sub(group):

    first = group[ group['renew_att_num'] == '1' ]
    
    if first.empty:
        group['first_attempt_date'] = ''
        group['first_day_of_month'] = 0
        group['first_transaction_hour'] = ''
        group['first_response_code'] = ''
        group['first_response_message'] = ''
        group['first_response_group'] = ''
        group['first_decline_type'] = ''
        group['days_between'] = 0
        return group
    
    last = group[ group['renew_att_num'] == group['renew_att_num'].max() ]
        
    first_attempt_date = first['transaction_date_in_string'].iloc[0]
    last_attempt_date = last['transaction_date_in_string'].iloc[0]
     
    
    last['first_attempt_date'] = first_attempt_date
    last['first_day_of_month'] = first['day_of_month'].iloc[0].astype(int)
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['first_transaction_hour'] = first['transaction_hour'].iloc[0]
    last['first_response_code'] = first['response_code'].iloc[0]
    
    first_response_msg = first['response_message'].iloc[0]
    last['first_response_message'] = first_response_msg  
    last['first_response_group'] =   first['response_message'].apply(group_response_msg).iloc[0]
    
    last['first_decline_type'] = decline_type(first_response_msg)
    last['days_between'] =  days_between(first_attempt_date, last_attempt_date)
   

    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)


# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)

time_start = time.time()        

date_condition = str(list(date_range_condition_values(start_date, end_date))).replace('[','').replace(']','')
site_date_condition = str(" received_date in (%s)" % date_condition)

udfValueToStatus = udf(convert_status, IntegerType())  

# print(site_date_condition)
sdf = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="cpg_transaction", keyspace="subs")\
    .load()
    
sdf =  sdf\
    .repartition(16, "subsegment_id")\
    .filter(site_date_condition)\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed'))")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", sdf["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()
    

grouped_schema = sdf.schema.add("first_attempt_date", StringType())\
                            .add("first_day_of_month", IntegerType())\
                            .add("first_transaction_hour", StringType())\
                            .add("first_response_code", StringType())\
                            .add("first_response_message", StringType())\
                            .add("first_response_group", StringType())\
                            .add("first_decline_type", StringType())\
                            .add("days_between", IntegerType()) 


@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            


pdf_all = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id")\
    .apply(merge_by_sub_spark)\
    .filter('days_between > 0')\
    .toPandas()
    
print("# total time:", time.time() - time_start)    
    
pdf_all

Py4JJavaError: An error occurred while calling o309.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: ShuffleMapStage 4 (toPandas at <ipython-input-3-6cdce6d4124b>:209) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: Failed to connect to /10.224.12.44:24886 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:519) 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:450) 	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:61) 	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434) 	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440) 	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408) 	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:32) 	at org.apache.spark